In [2]:
# Import pandas 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


# Read in white wine data 
white = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=';')

# Read in red wine data 
red = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')

In [3]:
# Add `type` column to `red` with value 1
red['type'] = 1

# Add `type` column to `white` with value 0
white['type'] = 0

# Append `white` to `red`
wines = red.append(white, ignore_index=True)

In [4]:
y = wines.quality
X = wines.drop(['quality', 'residual sugar', 'free sulfur dioxide', 'type'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=46, stratify=y)

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train_scaled = scaler.transform(X_train)

# Scale the test set
X_test_scaled = scaler.transform(X_test)

In [5]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras import metrics

from keras.utils import plot_model
#from kt_utils import *
import keras.backend as K
from sklearn import preprocessing, model_selection 

Using TensorFlow backend.


In [6]:
def create_model():
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu'))
    #layer 2
    model.add(Dense(50, activation='relu'))
    #layer 4
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [100,150,200,250,300]
epochs = [5,10,20,40,60,80,100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [7]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (abs(mean), stdev, param))

Best: -0.521120 using {'batch_size': 100, 'epochs': 100}
2.407693 (0.437532) with: {'batch_size': 100, 'epochs': 5}
1.383222 (0.107602) with: {'batch_size': 100, 'epochs': 10}
0.785371 (0.036050) with: {'batch_size': 100, 'epochs': 20}
0.573435 (0.015241) with: {'batch_size': 100, 'epochs': 40}
0.536456 (0.023081) with: {'batch_size': 100, 'epochs': 60}
0.526839 (0.020632) with: {'batch_size': 100, 'epochs': 80}
0.521120 (0.018569) with: {'batch_size': 100, 'epochs': 100}
3.343227 (0.786253) with: {'batch_size': 150, 'epochs': 5}
2.010202 (0.132539) with: {'batch_size': 150, 'epochs': 10}
1.162233 (0.079046) with: {'batch_size': 150, 'epochs': 20}
0.615142 (0.058519) with: {'batch_size': 150, 'epochs': 40}
0.561602 (0.010057) with: {'batch_size': 150, 'epochs': 60}
0.543537 (0.020609) with: {'batch_size': 150, 'epochs': 80}
0.539558 (0.010136) with: {'batch_size': 150, 'epochs': 100}
5.647461 (2.471679) with: {'batch_size': 200, 'epochs': 5}
2.108881 (0.352540) with: {'batch_size': 200

In [9]:
#Use scikit-learn to grid search the number of neurons
def create_model(neurons=1):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='uniform'))
    #layer 2
    model.add(Dense(neurons, activation='relu', kernel_initializer='uniform'))
    model.add(Dropout(0.5))
    #layer 3
    model.add(Dense(1, activation='linear', kernel_initializer='uniform'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=100, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 20, 40, 60, 80, 100]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [10]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.550257 using {'neurons': 100}
-3.886984 (2.029062) with: {'neurons': 1}
-1.323703 (0.149754) with: {'neurons': 5}
-0.855838 (0.141245) with: {'neurons': 10}
-0.637310 (0.026796) with: {'neurons': 20}
-0.578976 (0.003007) with: {'neurons': 40}
-0.564392 (0.014132) with: {'neurons': 60}
-0.553587 (0.023066) with: {'neurons': 80}
-0.550257 (0.010495) with: {'neurons': 100}


In [11]:
def create_model(hidden_layers=1):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='uniform'))
    
    #hidden layers
    for i in range(hidden_layers):
        # Add one hidden layer
        model.add(Dense(100, activation='relu',  kernel_initializer='uniform'))
    
    #layer 4
    model.add(Dense(1, activation='linear',  kernel_initializer='uniform'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, batch_size=100, verbose=0, epochs=100)
# define the grid search parameters
hidden_layers = [1,2,3,4]
param_grid = dict(hidden_layers = hidden_layers)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)


In [12]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.518887 using {'hidden_layers': 4}
-0.522053 (0.020351) with: {'hidden_layers': 1}
-0.535810 (0.013816) with: {'hidden_layers': 2}
-0.524399 (0.016212) with: {'hidden_layers': 3}
-0.518887 (0.018719) with: {'hidden_layers': 4}


In [13]:
# Use scikit-learn to grid search the activation function
def create_model(activation='relu'):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='uniform'))
    #layer 3
    model.add(Dense(100, activation=activation, kernel_initializer='uniform'))
    #layer 4
    model.add(Dense(1, activation='linear', kernel_initializer='uniform'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=100, verbose=0)
# define the grid search parameters
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.521276 using {'activation': 'relu'}
-0.521276 (0.018795) with: {'activation': 'relu'}
-0.545329 (0.021799) with: {'activation': 'tanh'}
-0.542919 (0.017470) with: {'activation': 'sigmoid'}
-0.548575 (0.023807) with: {'activation': 'hard_sigmoid'}
-0.539176 (0.014152) with: {'activation': 'linear'}


In [24]:
from keras.optimizers import Adam
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu'))
    #layer 2
    model.add(Dense(100, activation='relu'))
    #layer 3
    model.add(Dense(1, activation='linear'))
    optimizer = Adam(lr=learn_rate)
    # Compile model
    model.compile(optimizer = optimizer, loss='mean_squared_error', metrics=['accuracy'])
    return model

import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=100, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [25]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.526252 using {'learn_rate': 0.001}
-0.526252 (0.021165) with: {'learn_rate': 0.001}
-0.533763 (0.017367) with: {'learn_rate': 0.01}
-0.541151 (0.013768) with: {'learn_rate': 0.1}
-0.715966 (0.086021) with: {'learn_rate': 0.2}
-0.767468 (0.029794) with: {'learn_rate': 0.3}
